In [1]:
import lusid
import lusid.models as models
import os
from msrest.authentication import BasicTokenAuthentication

api_url = os.getenv("FBN_LUSID_API_URL")
api_token = {"access_token": os.environ["ACCESS_TOKEN"]}
credentials = BasicTokenAuthentication(api_token)
client = lusid.LUSIDAPI(credentials, api_url)

In [8]:
import uuid
from datetime import datetime
import pytz
from collections import namedtuple

def assert_response_is_not_error(expected_type, response):

    if isinstance(response, models.ErrorResponse):
        self.fail(response.detailed_message)
    elif isinstance(response, expected_type):
        return response
    else:
        self.fail("unknown response: {0}".format(type(response)))
        
def build_trade(trade_spec):
    return models.TradeDto(
        trade_id=str(uuid.uuid4()),
        type="StockIn",
        security_uid=trade_spec.id,
        settlement_currency="GBP",
        trade_date=trade_spec.trade_date,
        settlement_date=trade_spec.trade_date,
        units=100,
        trade_price=trade_spec.price,
        total_consideration=100 * trade_spec.price,
        source="Client")    

In [9]:
# create the portfolio

scope = "finbourne"
guid = str(uuid.uuid4())
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

request = models.CreatePortfolioRequest("portfolio-{0}".format(guid), "id-{0}".format(guid), "GBP", effective_date)
result = client.create_portfolio(scope, request)

assert_response_is_not_error(models.PortfolioDto, result)
assert(result.id.code == request.code)

portfolio_id = result.id.code

In [10]:
# add initial batch of trades

TradeSpec = namedtuple('TradeSpec', 'id price trade_date')
trade_specs = [
    TradeSpec("FIGI_BBG001S7Z574", 101, datetime(2018, 1, 1, tzinfo = pytz.utc)),
    TradeSpec("FIGI_BBG001SRKHW2", 102, datetime(2018, 1, 2, tzinfo = pytz.utc)),
    TradeSpec("FIGI_BBG000005547", 103, datetime(2018, 1, 3, tzinfo = pytz.utc))
]
trade_specs.sort(key=lambda ts: ts.id)

new_trades = list(map(build_trade, trade_specs))
add_trades_result = client.upsert_trades(scope, portfolio_id, new_trades)
initial_result = assert_response_is_not_error(models.UpsertPortfolioTradesDto, add_trades_result)

as_at_batch1 = initial_result.version.as_at_date

In [12]:
# add trade for 2018-1-8
trade = build_trade(TradeSpec("FIGI_BBG001S61MW8", 104, datetime(2018, 1, 8, tzinfo=pytz.utc)))
later_result = client.upsert_trades(scope, portfolio_id, [trade])
later_trade = assert_response_is_not_error(models.UpsertPortfolioTradesDto, later_result)

as_at_batch2 = later_trade.version.as_at_date

In [14]:
# add back dated trade
trade = build_trade(TradeSpec("FIGI_BBG001S6M3Z4", 105, datetime(2018, 1, 5, tzinfo=pytz.utc)))
backdated_result = client.upsert_trades(scope, portfolio_id, [trade])
backdated_trade = assert_response_is_not_error(models.UpsertPortfolioTradesDto, backdated_result)

as_at_batch3 = backdated_trade.version.as_at_date

In [16]:
def print_trades(trades):
    for trade in trades:
        print("{0}\t{1}\t{2}\t{3}\t{4}".format(trade.security_uid,
                                               trade.trade_date,
                                               trade.units,
                                               trade.trade_price,
                                               trade.total_consideration))

In [21]:
# get the list of trades
trades_result = client.get_trades(scope, portfolio_id, as_at=as_at_batch1)
trades_list = assert_response_is_not_error(models.VersionedResourceListTradeDto, trades_result)

assert(len(trades_list.values) == 3)
print("trades at {0}".format(as_at_batch1))
print_trades(trades_list.values)

trades at 2018-05-19 09:18:00.411910+00:00
FIGI_BBG001S7Z574	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG001SRKHW2	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000005547	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0


In [22]:
all_trades = client.get_trades(scope, portfolio_id, as_at=as_at_batch2)
trades_list = assert_response_is_not_error(models.VersionedResourceListTradeDto, all_trades)

assert(len(trades_list.values) == 4)
print("trades at {0}".format(as_at_batch2))
print_trades(all_trades.values)

trades at 2018-05-19 09:18:38.887002+00:00
FIGI_BBG001S7Z574	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG001SRKHW2	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000005547	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG001S61MW8	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [23]:
all_trades = client.get_trades(scope, portfolio_id, as_at=as_at_batch3)
trades_list = assert_response_is_not_error(models.VersionedResourceListTradeDto, all_trades)

assert(len(trades_list.values) == 5)
print("trades at {0}".format(as_at_batch3))
print_trades(all_trades.values)

trades at 2018-05-19 09:18:54.618102+00:00
FIGI_BBG001S7Z574	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG001SRKHW2	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000005547	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG001S6M3Z4	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
FIGI_BBG001S61MW8	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [24]:
all_trades = client.get_trades(scope, portfolio_id)
assert_response_is_not_error(models.VersionedResourceListTradeDto, all_trades)

print("trades at {0}".format(datetime.utcnow()))
print_trades(all_trades.values)

trades at 2018-05-19 09:22:51.771367
FIGI_BBG001S7Z574	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG001SRKHW2	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000005547	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG001S6M3Z4	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
FIGI_BBG001S61MW8	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0
